# Task: Text generation

Understand to text generation model, to build yourself dataset or take dataset from webinar and teach text generation model

In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
from keras.models import Sequential, Model
from keras.layers import GRU, LSTM, Embedding, Dense
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint

In [2]:
with open('evgenyi_onegin.txt', 'r') as file:
    text = file.read()
print(f'Length of text: {len(text)} characters')

Length of text: 286984 characters


In [3]:
print(text[:1000])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
                        Бессонниц, легких вдохновений,
                        Незрелых и увядших лет,
                        Ума холодных наблюдений
                        И сердца горестных замет.

            

# Preparing

In [4]:
text = text + text
vocab = sorted(set(text))
print(f'The unique characters is {len(vocab)}')

The unique characters is 131


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [6]:
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print(text_as_int)
print(len(text_as_int))
print(len(text))

[ 71 110 104 ... 104 121   0]
573968
573968


# Training

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(9):
    print(idx2char[i.numpy()])

А
л
е
к
с
а
н
д
р


In [9]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [12]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10_000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [13]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

In [14]:
model = Sequential([
        Embedding(vocab_size, embedding_dim),
        LSTM(rnn_units, return_sequences=True),
        Dense(vocab_size)])

In [15]:
class RNNgenerator(Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()

        self.emb = Embedding(vocab_size, embedding_dim)

        self.gru1 = GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')

        self.fc = Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1) / 2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim,
                batch_input_shape=[batch_size, None]),

        LSTM(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),

        LSTM(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),

        LSTM(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),

        LSTM(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),

        Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
                  vocab_size=len(vocab),
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 128)           16768     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          4722688   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_4 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30051843 (114.64 MB)
Trainable params: 3

In [20]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 1.3410777e-05,  3.9764172e-06,  8.9904961e-06, ...,
        -1.0529433e-05,  3.0581878e-06,  5.5118726e-06],
       [ 3.7017799e-05,  6.5898244e-06,  2.7266677e-05, ...,
        -1.6803517e-05,  2.6158745e-05,  1.3609933e-05],
       [ 6.4786829e-05,  1.5839796e-05,  5.7312918e-05, ...,
        -1.0970036e-05,  6.0849765e-05,  2.3962588e-05],
       ...,
       [-3.0246535e-03, -2.0020348e-03,  2.4522289e-03, ...,
         1.4883170e-03,  6.9009094e-04,  8.7924981e-03],
       [-3.0445270e-03, -2.0059932e-03,  2.4410463e-03, ...,
         1.3925884e-03,  5.4525305e-04,  8.9247655e-03],
       [-3.0567581e-03, -1.9996858e-03,  2.4180394e-03, ...,
         1.3049056e-03,  3.9138179e-04,  9.0270815e-03]], dtype=float32)>

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("\nNext char predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'о б (не правда ли?) пора!\n\n                                     XLIX\n\n                        Кто б '

Next char predictions: 
 'ЖэъФЬ"ш ЦwЛ9IP8зоч{ЕДПъЦ2NтIдYр7t5 Г}иRTяA;Ю6DЮhоRmЬиЖAнmd"щХHpeКAьБ-luЭ Кc9rП 060НПшыcИ4M2spСNQфBуb'


# Train model

In [23]:
def loss(labels, logits):
    return sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
Scalar_loss:       4.8758526


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88 * 5,
    save_weights_only=True)

In [26]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
88/88 [==============================] - 42s 381ms/step - loss: 2.3651
Epoch 2/200
88/88 [==============================] - 38s 410ms/step - loss: 2.0201
Epoch 3/200
88/88 [==============================] - 36s 392ms/step - loss: 1.7007
Epoch 4/200
88/88 [==============================] - 36s 398ms/step - loss: 1.5349
Epoch 5/200
88/88 [==============================] - 37s 411ms/step - loss: 1.4351
Epoch 6/200
88/88 [==============================] - 37s 403ms/step - loss: 1.3858
Epoch 7/200
88/88 [==============================] - 36s 401ms/step - loss: 1.3409
Epoch 8/200
88/88 [==============================] - 36s 400ms/step - loss: 1.3206
Epoch 9/200
88/88 [==============================] - 36s 403ms/step - loss: 1.3214
Epoch 10/200
88/88 [==============================] - 40s 444ms/step - loss: 1.2892
Epoch 11/200
88/88 [==============================] - 37s 403ms/step - loss: 1.3141
Epoch 12/200
88/88 [==============================] - 37s 404ms/step - loss: 1.2706
E

# Generate text

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 128)            16768     
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           4722688   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_8 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30051843 (114.64 MB)
Trainable params: 3

In [37]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1010

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [39]:
text_ = generate_text(model, start_string="Я помню чудное мгновенье")
print(text_)
# print(len(text_))

Я помню чудное мгновенье
                        Не ты ли, милое виденье,
                        В прозрачной темноте мелькнул,
                        Приникнул тихо к изголовью?
                        Не ты ль, с отрадой и любовью,
                        Слова надежды мне шепнул?
                        Кто ты, мой ангел ли хранитель,
                        Или коварный искуситель:
                        Мои сомненья разреши.
                        Быть может, это все пустое,
                        Обман неопытной души!
                        И суждено совсем иное...
                        Но так и быть! Судьбу мою
                        Отныне я тебе вручаю,
                        Перед тобою слезы лью,
                        Твоей защиты умоляю...
                        Вообрази: я здесь одна,
                        Никто меня не понимает,
                        Рассудок мой изнемогает,
                        И молча готовым он полунет,
                        Без п